In [ ]:
from keras.datasets import fashion_mnist
dataset= fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = dataset

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
print(X_train.shape)
print(y_train.shape)
 
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
